In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:92% !important; }</style>"))

%load_ext autoreload
%autoreload 2
import sys, os
from os.path import expanduser
## actions required!!!!!!!!!!!!!!!!!!!! change your folder path 
path = "~/Documents/G3/MA-prediction"
path = expanduser(path)
sys.path.append(path)

import data_science_MA_kit as dsk
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm
# import wrds
import re

pd.options.mode.chained_assignment = None

# Data Processing Notebooks Outline: 

We download raw M&A deals data from SDC platinum, and process them in several notebooks (in order):
- Notebook 0: basic cleaning.
- Notebook 1 & 2: variable transformation & merge with CRSP database.
- Notebook 3: applying filters.

General guidelines for these data processing notebooks:
- We create new columns (variables)  on all the rows first, before applying  any  filters. 
- When filtering we should not drop any row directly, in case we want to retrieve them later. Instead we add another column called `retain` to indicate whether to retain the row after applying the filters. 
- From time to time we save the intermediate result as an `hdf` file, as some codes (especially those querying the CRSP database) need tens of minutes to run. Thus we want to run it for just once and store the results. The advantage of `hdf` over `csv` is that it preserves data type like `datetime.date`. Only when we need to inspect the dataset by `Excel` or `Numbers` shall we save it as `csv`.
- These notebooks shall be highly modular, meaning that almost every data operation should be encapsulated in a function in the helper package. Each function is developed in another individual notebook (thus tens of development notebooks). In this way the end user only needs to read the comment without digging into the codes.

## Data preprocessing 0: Basic Cleaning
Specifically in this notebook we will do the following:

- Load column names from the report file. Load raw data. Change column names. 
- Transform date-like columns to `datetime.date` dtype. Transform float-like columns to float.
- Correct `consid` for some deals manually.
- Fill missing `pr_initial` by `pr`.

# Load data
## Load column names
Load column names from the report file. 

In [2]:
filepath = f"{path}/data/report.rpt"
# extract colnames from report file
col_names = dsk.extract_colnames_from_report_file(filepath)
col_names

['master_deal_no',
 'statc',
 'one_day',
 'aone_day',
 'dao',
 'da',
 'dateannorig_days',
 'de',
 'dateeffexp',
 'dw',
 'definitive_agt',
 'da_date',
 'dateval',
 'dcom',
 'dcomeff',
 'att',
 'attitude_change_yn',
 'initial_rec',
 'tn',
 'ttic',
 'tcu',
 'texch',
 'an',
 'apub',
 'atic',
 'acu',
 'anatc',
 'aexch',
 'cross',
 'ttf_macro_desc',
 'ttf_mid_desc',
 'atf_macro_desc',
 'atf_mid_desc',
 'val',
 'mv',
 'amv',
 'pr',
 'pr_initial',
 'valamend',
 'pr4wk',
 'pr1wk',
 'pr1day',
 'tprday',
 'tpr1daya',
 'tpr1wka',
 'tpr4wka',
 'ppmday',
 'ppmwk',
 'ppm4wk',
 'ac4wk',
 'ac1wk',
 'ac1day',
 'aprday',
 'apr1daya',
 'apr1wka',
 'apr4wka',
 'consid_struct_desc',
 'consid',
 'consido',
 'consids',
 'cha',
 'tend',
 'phda',
 'psought',
 'term',
 'termfpct',
 'synop',
 'hdate',
 'hosthprice',
 'hval',
 'hevent',
 'hosthval',
 'competecode',
 'competeval',
 'lbo',
 'afinancial',
 'alp',
 'aspv',
 'awk',
 'hedge_fund_involv_yn',
 'collar',
 'debtrat',
 'roe',
 'eps',
 'pe',
 'pricebook',
 'e

## Load data & Change column names.
Load data from the `csv` file. Change column names. 

In [3]:
# load data
filepath = f"{path}/data/df.csv"
df = pd.read_csv(filepath, index_col=0, na_values=['nm', 'np'], low_memory=False)

# change column names
df.index.name = col_names[0]
df.columns = col_names[1:]

dsk.print_shape(df)
df.tail()

The dataset is of size (9854, 94).


,statc,one_day,aone_day,dao,da,dateannorig_days,de,dateeffexp,dw,definitive_agt,...,pricebook,eqvalcf,eqvalsales,eqval,tlia,cass,clia,lockup,dae,vest
master_deal_no,,,,,,,,,,,,,,,,,,,,,
3846599020,P,12/14/21,12/14/21,12/15/21,12/15/21,0,NaN,12/31/22,NaN,Yes,...,6.389,16.389,4.750,5699.204,"1,265.1",535.0,205.9,No,No,No
3847933020,C,12/16/21,NaN,12/17/21,12/17/21,0,05/13/22,06/30/22,NaN,Yes,...,5.573,59.340,5.089,2456.130,347.3,237.8,147.5,No,No,No
3848920020,C,07/09/21,12/17/21,07/11/21,12/19/21,161,03/31/22,04/01/22,NaN,Yes,...,1.346,13.529,0.621,784.682,400.0,433.0,201.0,No,No,No
3847595020,C,12/15/21,12/17/21,12/16/21,12/20/21,4,06/08/22,06/06/22,NaN,No,...,7.331,20.551,4.971,28373.205,"3,703.3","2,440.2","1,551.3",No,No,No
3851185020,C,NaN,12/22/21,12/23/21,12/23/21,0,05/27/22,05/27/22,NaN,Yes,...,14.716,11.498,2.984,6082.266,"2,905.9",943.8,487.7,No,No,No


# Transform date-like and float-like columns
Transform date-like columns to `datetime.date` dtype. Transform float-like columns to float.

In [4]:
# transform date-like columns
cols_dt = ['one_day', 'aone_day', 'dao', 'da', 'de', 'dateeffexp', 'dw', 'da_date', 'dateval', 'dcom', 'dcomeff']

# apply function to the columns
df[cols_dt] = df[cols_dt].apply(dsk.convert_date_str_ser_to_datetime)

# transform numeric-like columns
cols_float = ['val', 'mv', 'amv', 'pr', 'ppmday', 'ppmwk', 'ppm4wk', 'roe', 'tlia', 'cass', 'clia']

# apply function to the columns
df[cols_float] = df[cols_float].apply(dsk.convert_num_str_ser_to_float)

# Correct `consido` for some deals manually
Correct `consid` for some deals manually.

In [5]:
# correct data errors
cols = ['consid', 'consido']
df[cols] = dsk.correct_consid(df[cols])

# Fill missing `pr_initial` by `pr`

In [6]:
# fill missing `pr_initial` by `pr`
df.pr_initial[df.pr_initial.isna()]=df.pr[df.pr_initial.isna()]

# save intermediate result

In [7]:
filepath = f"{path}/data/df_basic_cleaning.h5"

df.to_hdf(filepath, key = 'df')

/var/folders/l5/gqn1skjd0v366h5y3kbms6lh0000gn/T/ipykernel_77522/806000311.py:3: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['statc', 'one_day', 'aone_day', 'dao', 'da', 'de', 'dateeffexp', 'dw',
       'definitive_agt', 'da_date', 'dateval', 'dcom', 'dcomeff', 'att',
       'attitude_change_yn', 'initial_rec', 'tn', 'ttic', 'tcu', 'texch', 'an',
       'apub', 'atic', 'acu', 'anatc', 'aexch', 'cross', 'ttf_macro_desc',
       'ttf_mid_desc', 'atf_macro_desc', 'atf_mid_desc', 'valamend',
       'consid_struct_desc', 'consid', 'consido', 'consids', 'cha', 'tend',
       'term', 'synop', 'hdate', 'hosthprice', 'hval', 'hevent', 'hosthval',
       'competecode', 'competeval', 'lbo', 'afinancial', 'alp', 'aspv', 'awk',
       'hedge_fund_involv_yn', 'collar', 'lockup', 'dae', 'vest'],
      dtype='object')]

  df.to_hdf(filepath, key = 'df')
